# Task 4.9
# Part 1

## Step 3 - Import your analysis libraries, as well as your new customer data set as a dataframe


In [1]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy


In [7]:
# Path
path = r'C:\\Users\\mshhan\\Documents\\05-2024 Instacart Basket Analysis'

# Import new customer data
customers = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))



In [8]:
# Checking new data
customers.head()


,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [9]:
customers.shape


(206209, 10)

In [10]:
customers.describe()


,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [11]:
customers.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


## Step 4 - Wrangle the data so that it follows consistent logic
### Rename the variables to bring similar format


In [12]:
# Rename First Name as first_name
customers.rename(columns = {'First Name' : 'first_name'}, inplace = True)

# Rename Surnam as last_name
customers.rename(columns = {'Surnam' : 'last_name'}, inplace = True)

# Rename Gender as gender
customers.rename(columns = {'Gender' : 'gender'}, inplace = True)

# Rename STATE as state
customers.rename(columns = {'STATE' : 'state'}, inplace = True)

# Rename Age as age
customers.rename(columns = {'Age' : 'age'}, inplace = True)

# Rename fam_status as marital_status
customers.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)


In [13]:
# Checking
customers.columns


Index(['user_id', 'first_name', 'last_name', 'gender', 'state', 'age',
       'date_joined', 'n_dependants', 'marital_status', 'income'],
      dtype='object')

6 variables were renamed and now all the columns have a similar form

For now I dont think I need to drop any column. Maybe for data protection reasons I believe I could drop first_name and last_name.


## Step 5 - Complete the fundamental data quality and consistency checks you’ve learned throughout this Achievement


In [17]:
# Checking missing values
customers.isnull().sum()


user_id               0
first_name        11259
last_name             0
gender                0
state                 0
age                   0
date_joined           0
n_dependants          0
marital_status        0
income                0
dtype: int64

There are 11,259 missing first names. This is not a key variable for the analysis, so no need to address them.


In [18]:
# Checking duplicates
df_dups = customers[customers.duplicated()]
df_dups


,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income


There are no duplicates in the dataframe.

In [19]:
# Checking mixed-type
customers.dtypes


user_id            int64
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
n_dependants       int64
marital_status    object
income             int64
dtype: object

In [20]:
for col in customers.columns.tolist():
  weird = (customers[[col]].map(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len(customers[weird]) > 0:
    print(col)


first_name


`first_name column` has mixed data type. Let's fix it.


In [21]:
# Address string data type to first name
customers['first_name'] = customers['first_name'].astype('str')


In [22]:
# Let's check again the data type
customers.dtypes


user_id            int64
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
n_dependants       int64
marital_status    object
income             int64
dtype: object

In [23]:
# Let's check again the mixed data-type
for col in customers.columns.tolist():
  weird = (customers[[col]].map(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len(customers[weird]) > 0:
    print(col)


There is no mixed data type columns and all the columns are the right data type.


## Step 6 - Combine your customer data with the rest of your prepared Instacart data


In [25]:
# Path to the prepared Instacart data
ords_prods_file = os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_updated.pkl')

# Check if the file exists and load it
if os.path.exists(ords_prods_file):
    # Import the rest of prepared Instacart data
    ords_prods_merge = pd.read_pickle(ords_prods_file)
else:
    print(f"File not found: {ords_prods_file}")


In [26]:
# Checking data type
ords_prods_merge.dtypes


order_id                      int64
user_id                       int64
order_number                  int64
day_of_week                   int64
order_hour_of_day             int64
days_since_prior_order      float64
product_id                    int64
add_to_cart_order             int64
reordered                     int64
_merge_existing            category
product_name                 object
aisle_id                      int64
department_id                 int64
prices                      float64
_merge                     category
price_label                  object
busiest_day                  object
busiest_days_updated         object
max_order                     int64
loyalty_flag                 object
avg_price                   float64
spending_flag                object
median_days_since_prior     float64
order_frequency_flag         object
dtype: object

The key columns are `user_id` and both are the same data type.

In [27]:
# Drop the '_merge' column from the previous exercise if it exists
if '_merge' in ords_prods_merge.columns:
    ords_prods_merge = ords_prods_merge.drop(columns=['_merge'])


In [28]:
# Merge ords_prods_merge with customers data
ords_prods_customers_merge = ords_prods_merge.merge(customers, on='user_id', indicator=True)


In [29]:
# Step 8 - Export this new dataframe as a pickle file
ords_prods_customers_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_customers_aggregated.pkl'))
